# SYS 6014 Final project report
## Introduction
Long-short term memory(LSTM) is appealing to time series analysis because of its ability to learn the time dependency in data and the superior performance. However most of the research focused on the performance evaluation but paid less attention to the application of result. Therefore, this study discuss the application of using the result of LSTM to build a statistical decision model by following the reproducible principle.   
Note: the pdf written by jupyter notebook cannot show images. Please check out the jupyter notebook for more information. The figure, LSTM model and results can directly be generated by the program under code directory.

## Background
The motivation of this study was to build a binary classification model according to Predictive Maintenance Template (Predictive Maintenance Template, 2015). It enables engineers to maintain the assets, including which the aircraft engine, machine, turbines, etc., in advance by the prediction result of a model.  
The importance of the decision in the context of this study can be explained by the Framing biases in human decision making process. When facing the potential failure risk, there are two kinds of losses, which are defined as payoffs in this study. However, human operators who in charges of the operation of an asset may take risks to continue operated the machine to avoid huge asset suspension fare, with the risk of spending larger further maintenance fare after asset failure by making false decision. A decision analysis model may help human operators realize the utilities of his decisions and therefore minimize the potential loss.   
The dataset is available on the internet, which records the information of aircraft engine use life. The function of prediction component: a binary classification tell us whether or not the engine will fail in 30 cycles. By the great performance of deep learning models, this study choose Long-short term memory(LSTM) to predict time series. After the training and test of LSTM complete. The decision analysis component will be finished by using a decision tree analysis. The trade off between using the 'empirical' conjecture or using the LSTM model to predict will be discussed in the following content. 

## Prediction component 
### Dataset
The prediction component is achieved by LSTM model. The procedure follows the common procedue in deep learning and the predictive maintenance template. The description of data set is shwon in Figure 1.

In [13]:
%%html
<figure><img src="pictures/dataset.png" width="80%" align="middle" >
<figcaption>Figure 1. dataset description</figcaption></figure>

The dataset describes the aircraft engine operation state until it fails. The indicator of using life is measured by the second attribute: cycle. Therefore for each engine id: the maximum value of its corresponding cycle is the maximum useful life.   
This study realize the binary classification model, it predicts whether the engine will fail in certain cycles. The window is set to 30.   
### Data preprocessing
Then the raw data will be read from the Data directory. Then it will be normalized and labeled. The same processing procedure will be apllied for both training and test data.
### Model Construction
The data will firstly be transformed into the format that fits keras LSTM layer, then the model will be constructed. Figure 2 shows the summary of the model. The window for LSTM is set to 50.

In [22]:
%%html
<figure><img src="pictures/model_summary.PNG" width="70%" align="middle" >
<figcaption>Figure 2. Model summary</figcaption></figure>

### Model Training
The model will be trained for 100 epoach, with bath_size 200.  The loss and accuracy during the training procedure is shown in Figure 3 and Figure 4.

In [23]:
%%html
<figure><img src="pictures/model_accuracy.png" width="70%" align="middle" >
<figcaption>Figure 3. Accuracy plot</figcaption></figure>
<figure><img src="pictures/model_loss.png" width="70%" align="middle" >
<figcaption>Figure 4.Loss plot</figcaption></figure>

### Model evaluation
For training: precision =  0.9690553745928339  recall =  0.9596774193548387   
For test: Precision:  0.9583333333333334 Recall:  0.92  F1-score: 0.9387755102040817   
It is proved by the above criteria that the LSTM model is capable of leanring and predicting the time series in this study.

## Decision model
As introduced previously, when the operator or manager is given the information of potential risk, some may choose to suspend the engine, some may let it continue operating. Such information can be given by previous experience, or by the prediction model achieved above. Therefore there are two different decision tree to be considered. For further analysis, other terms will be described below.
### Assumptions:
1. In reality, the RUL is not known. But we assume it can be predicted as well (through regression), and the difference between the real RUL and predicted RUL can be neglected for classification study like this.
2. The decision starts at the cycle when it remaines w0 cycles. 
3. For prior belief, the probability remains the same within wp cycles. wp > w0 = 30.
### Prior beliefs
In this project, the prior belief is the probability that the engine will fail in wp cycles, wp > w0 =30. This is given by previous experience.
### Action Set
$$\mathbb A = \{A_{s} :stop operation, A_{c}:continue operation\}$$
### State Space
then state space can be described as: $$\mathbb X = \{X_{n}:Operational with in w0, X_{f}:malfunction within w0\}$$
### Data generation process
The predicted sequence of labels indicating the potential risk within w0 is generated by LSTM. Which can be found in pictures directory. 
### Parameter space
$$\Theta = \mathbb R \times [argmin\{attributes\},argmax\{attributes\})$$ for each of attributes of data set.
### Payoffs
It can be described as: the expected payoff can be described as the economic based on the decsions made by the given potential risk information.
$$\textbf{E}(A_{s}|X_{n}) = 3000 , \textbf{E}(A_{s}|X_{f}) = 1000, 
\textbf{E}(A_{c}|X_{n}) = 0,  
\textbf{E}(A_{c}|X_{f}) = 2000$$   
Those values are all 'negative', because they are costs.
### Utility Funtion
$$\textbf{U} = \sum \textbf{E}(A|X) \times P(A|X)$$
## Decision Tree analysis

### part one:
If the decision maker relies on prior belief, and we assume the suspension: $$A_{s}$$ is the best action. 

%%html
<figure><img src="pictures/prior_belief.png" width="70%" align="middle" >
<figcaption>Figure 5. decision analysis 1</figcaption></figure>

By analysis, we have $$2000b+0-1000b+3000+300b>0$$   
it euals to :$$b>0.75$$ which means that the prior belief(probability of failure) must be larger than 0.75.
### part two
If the decision maker relies on LSTM model, and we assume the suspension: $$A_{s}$$ is the best action. Then we introduce recall and precision in analysis. 

In [2]:
%%html
<figure><img src="pictures/pre_recall.png" width="70%" align="middle" >
<figcaption>Figure 6. decision analysis 2</figcaption></figure>

By analysis, we have $$2000pr + 0 + 2000(1-r)-1000pr-3000(1-p)r-1000(1-r)>0$$   
it euals to :$$r(1-p)<1/4$$ which means that both recall rate and precision must be high enough to meet the assumption.
### Discussion
If the prior belief does not meet the minimum value of being reliable, then it should not be used to make shut down decisions.   
Similarly, if the recall and precision of the LSTM model is too low, then it will face the same issue with prior beliefs.   
However, it is possible that in some cases, both the window for predictions and window size for LSTM could be impractical. Therefore we can discuss the performance of each decision tree in broader spectrumm or narrower spectrum.   
Furthermore, the prior beliefs can be updated using the previous prediction result of LSTM, which refers to probabilities.

### Further works
1. The classification problem can be extended to muticlassification problem, which means that we can make more decisions in a large window w0. Therefore there will be more information available for decision makers.   
2. The decision simplified the real situation and only consider a single moment decision making process.   
3. A more comprehensive statistical model is required for more analysis.   
## Conclusion
The study trained a LSTM neural network to realize the prediction of a time series data. And it analyzed in which conditions decision makers can do with the result of deep learning model predictions.